In [39]:

import os
import pandas as pd

import random

random.seed(42)

In [40]:
df = pd.read_csv('Redos_summery.csv')
df.head()

,model_name,prompt_type,temperature,token_length,Count,Sample Count Proportion,Rounded
0,GPT3.5,raw,0.0,128,31,0.842255,1
1,GPT3.5,raw,0.2,128,132,3.586375,4
2,GPT3.5,raw,0.4,128,66,1.793187,2
3,GPT3.5,raw,0.6,128,290,7.879157,8
4,GPT3.5,raw,0.8,128,295,8.015004,8


In [41]:
def parse_rescue(text, model_name, prompt_type, temperature, token_length):
    # Split the text into sections using "id:" as a delimiter

    sections = text.split('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    
    # Initialize a list to store the parsed data
    parsed_data = []
    
    # Iterate through each section (excluding the first empty section)
    for section in sections:
        # Initialize a dictionary to store the parsed information for this section
        section_data = {}
        
        current_data = section.split('\n')
        # print(current_data)
        if current_data[0] == '':
            current_data = current_data[1:]
        
        if len(current_data) < 3:
            continue
        
        section_data['model_name'] = model_name
        section_data['prompt_type'] = prompt_type
        section_data['temperature'] = temperature
        section_data['token_length'] = token_length
        
        # Extract the ID
        section_data['id'] = (int(current_data[0].split(":")[-1].strip())-1)//10

        section_data['regex_pattern'] = current_data[1].strip()
        
        section_data['status'] = current_data[2].strip()
        
        if section_data['status'] == "RESULT-TRUE":
            section_data['status'] = True
        else:
            section_data['status'] = False

        section_data['main_redos_pattern'] = current_data[3].strip()
        section_data['attack_string'] = current_data[5].strip().split("AttackString:")[-1][1:-1]
        section_data['sub_redos_pattern'] = current_data[6].strip().split("patternType: ")[-1]
        

        # Append the parsed data for this section to the list
        parsed_data.append(section_data)
    
    return parsed_data

In [42]:
files = os.listdir(f'./ReDoSHunterOutput')
files = [file for file in files if file.endswith('.txt') and '_true_' in file]
print(files)

['GPT3.5_Output_refined_0.2_128_only_redos_true_s_java_11111_0_2023_10_27_11_41_22.txt', 'GPT3.5_Output_raw_0.0_128_only_redos_true_s_java_11111_0_2023_10_26_22_15_22.txt', 'GPT3.5_Output_raw_0.4_128_only_redos_true_s_java_11111_0_2023_10_26_23_47_11.txt', 'GPT3.5_Output_raw_0.6_128_only_redos_true_s_java_11111_0_2023_10_27_00_57_23.txt', 'GPT3.5_Output_raw_1.0_128_only_redos_true_s_java_11111_0_2023_10_27_11_56_16.txt', 'Phi_Output_refined_0.2_128_only_redos_true_s_java_11111_0_2023_10_27_11_13_38.txt', 'GPT3.5_Output_raw_0.8_128_only_redos_true_s_java_11111_0_2023_10_27_02_09_47.txt', 'T5_Output_raw_0.6_128_only_redos_true_s_java_11111_0_2023_10_26_20_46_51.txt', 'Phi_Output_raw_0.6_128_only_redos_true_s_java_11111_0_2023_10_26_14_40_01.txt', 'GPT3.5_Output_refined_0.6_128_only_redos_true_s_java_11111_0_2023_10_27_12_25_23.txt', 'T5_Output_raw_0.2_128_only_redos_true_s_java_11111_0_2023_10_26_18_25_41.txt', 'Phi_Output_raw_0.8_128_only_redos_true_s_java_11111_0_2023_10_26_15_31_58.tx

In [43]:
all_data = []
sample_data = []
for file_name in files:
    model = file_name.split("_")[0]
    prompt_type = file_name.split("_")[2]
    temperature = file_name.split("_")[3]
    token_length = file_name.split("_")[4]
    sample_number = df.loc[(df['model_name'] == model) & (df['prompt_type'] == prompt_type) & (df['temperature'] == float(temperature)) & (df['token_length'] == int(token_length))]['Rounded'].values[0]
    with open(f'./ReDoSHunterOutput/{file_name}', 'r', encoding='latin-1') as f:
        data = f.read()
        data = "id:"+"id:".join(data.split('id:')[1:])
        parsed_data = parse_rescue(data, model, prompt_type, temperature, token_length)
        all_data += parsed_data
        parsed_data = random.sample(parsed_data, sample_number)
        sample_data += parsed_data
        

print(len(all_data))
print(len(sample_data))

28086
714


In [44]:
df = pd.DataFrame(all_data)
df.to_csv('all_data.csv', index=False)


In [45]:
df = pd.DataFrame(sample_data)
df.to_csv('sample_data.csv', index=False)